In [1]:
import os
import sys
sys.path.append('../')
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup the Worker

In [2]:
project = "pulpo-ammonia"
databases = ["nc-inventories-ei310-all", "ecoinvent-3.10-cutoff"]
methods = "('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')"

In [3]:
# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = r"C:\APPS\GAMS\win64\40.1\gams.exe"

In [4]:
pulpo_worker = pulpo.PulpoOptimizer(project, databases, methods, directory)
pulpo_worker.intervention_matrix="ecoinvent-3.10-biosphere"

In [5]:
pulpo_worker.get_lci_data()

### Get the Choices

In [6]:
choices_biogas = [
    "anaerobic digestion of animal manure, with biogenic carbon uptake",
    "anaerobic digestion of agricultural residues, with biogenic carbon uptake",
    "treatment of sewage sludge by anaerobic digestion, cut-off with biogenic carbon uptake",
    "treatment of industrial wastewater by anaerobic digestion, cut-off with biogenic carbon uptake",
    "treatment of biowaste by anaerobic digestion, cut-off with biogenic carbon uptake",
    "anaerobic digestion of sequential crop, with biogenic carbon uptake"
]

choices_hydrogen = [
    "hydrogen production, biomass gasification",
    "hydrogen production, biomass gasification, with CCS",
    "hydrogen production, steam methane reforming of biomethane",
    "hydrogen production, steam methane reforming of biomethane, with CCS",
    "hydrogen production, steam methane reforming of natural gas, with CCS",
    "hydrogen production, PEM electrolysis, green",
    "green hydrogen",
    "hydrogen production, plastics gasification",
    "hydrogen production, plastics gasification, with CCS"
]

choices_heat = [
    "heat from biomethane",
    "heat from biomethane, with CCS",
    "heat from hydrogen",
    "heat from natural gas, with CCS"
]

choices_ammonia = [
    "ammonia production, steam methane reforming of biomethane",
    "ammonia production, steam methane reforming of biomethane, with CCS",
    "ammonia production, steam methane reforming of natural gas, with CCS",
    "ammonia production, from nitrogen and hydrogen"
]

choices_biomethane = [
    "biogas upgrading to biomethane, chemical scrubbing",
    "biogas upgrading to biomethane, chemical scrubbing w/ CCS",
    "biogas upgrading to biomethane, membrane",
    "biogas upgrading to biomethane, membrane w/ CCS",
    "biogas upgrading to biomethane, pressure swing adsorption",
    "biogas upgrading to biomethane, pressure swing adsorption w/ CCS",
    "biogas upgrading to biomethane, water scrubbing",
    "biogas upgrading to biomethane, water scrubbing w/ CCS"
]

In [7]:
# Retrieve activities for each category
biogas_activities = pulpo_worker.retrieve_activities(activities=choices_biogas)
hydrogen_activities = pulpo_worker.retrieve_activities(activities=choices_hydrogen)
heat_activities = pulpo_worker.retrieve_activities(activities=choices_heat)
biomethane_activities = pulpo_worker.retrieve_activities(activities=choices_biomethane)

ammonia_activities = pulpo_worker.retrieve_activities(activities=choices_ammonia)
# Add BAU Ammonia from ecoinvent as choice
ammonia_activities.append(pulpo_worker.retrieve_activities(reference_products="ammonia, anhydrous, liquid", activities="ammonia production, steam reforming, liquid", locations="RER w/o RU")[0])

In [8]:
choices = {
    "biogas": {x: 1e10 for x in biogas_activities},
    "hydrogen": {x: 1e10 for x in hydrogen_activities},
    "heat": {x: 1e10 for x in heat_activities},
    "biomethane": {x: 1e10 for x in biomethane_activities},
    "ammonia": {x: 1e10 for x in ammonia_activities},
}

In [9]:
# Additional constraints
# Limit the amount of CCS
CCS_limit = pulpo_worker.retrieve_activities(activities="carbon dioxide storage and transport 200 km pipeline, storage 1000 m")
upper_limit = {CCS_limit[0]: 2}
lower_limit = {CCS_limit[0]: 1}

# Limit a random environmental intervention
radon_limit = pulpo_worker.retrieve_envflows(activities="Radon-222")
upper_elem_limit = {radon_limit[0]: 100}

### Demand

In [10]:
ammonia_market = pulpo_worker.retrieve_activities(activities="new market for ammonia")

In [11]:
demand = {ammonia_market[0]: 1}

### Instantiate and Solve

In [19]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_limit, lower_limit=lower_limit, upper_elem_limit=upper_elem_limit)

Creating Instance
Instance created


In [20]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\APPS\GAMS\win64\40.1\gams.exe

GAMS WORKING DIRECTORY: C:\Users\FLECHT~1\AppData\Local\Temp\tmp0gcpt99f



In [22]:
def extract_flows(instance, mapping, metadata, flow_type):
    """
    Extracts scaling factors or inventory flows from a Pyomo model instance.

    Parameters:
        instance (Pyomo model): The Pyomo model instance.
        mapping (dict): Maps process keys to IDs.
        metadata (dict): Maps IDs to metadata descriptions.
        flow_type (str): 'scaling' for process_map, 'intervention' for intervention_map.

    Returns:
        pd.DataFrame: Indexed by **ID**, containing **Key, Metadata, and Value**, sorted by Value (descending).
    """

    inverse_map = {v: k for k, v in mapping.items()}  # Reverse lookup
    
    # Select correct flow variable ('scaling' or 'intervention')
    flows = instance.scaling_vector if flow_type == 'scaling' else instance.inv_flows if flow_type == 'intervention' else None
    if flows is None:
        raise ValueError("Invalid flow_type. Use 'scaling' or 'intervention'.")

    # Retrieve data from flows
    data = {'ID': [], 'Key': [], 'Metadata': [], 'Value': []}
    for flow in flows:
        data['ID'].append(flow)
        data['Key'].append(inverse_map.get(flow, 'Unknown'))
        data['Metadata'].append(metadata.get(flow, 'No Metadata'))
        data['Value'].append(flows[flow].value)

    return pd.DataFrame(data).set_index('ID').sort_values('Value', ascending=False)

### Test the function
extract_flows(pulpo_worker.instance, pulpo_worker.lci_data['process_map'], pulpo_worker.lci_data['process_map_metadata'], flow_type='scaling')
#extract_flows(pulpo_worker.instance, pulpo_worker.lci_data['intervention_map'], pulpo_worker.lci_data['intervention_map_metadata'], flow_type='intervention')

,Key,Metadata,Value
ID,,,
23593,"(nc-inventories-ei310-all, 7bdc8ab4c3ab9f8d8fccdf1b37cca850)",heat from biomethane | heat from biomethane | GLO,13.952525
10040,"(ecoinvent-3.10-cutoff, 498a0521f9d3349797fbdff029ccc6c1)","market for heat, central or small-scale, natural gas | heat, central or small-scale, natural gas | RoW",3.580903
23531,"(nc-inventories-ei310-all, fc378c5b9e61e417e77ba0c166897da5)","carbon dioxide storage and transport 200 km pipeline, storage 1000 m | carbon dioxide storage and transport 200 km pipeline, storage 1000 m | GLO",2.000000
23581,"(nc-inventories-ei310-all, 8edab54f6d0f63e481613471626b8777)","market for biogas, sustainable feedstocks | biogas | GLO",1.808420
23574,"(nc-inventories-ei310-all, ee52fb3612dc8d0407672db32f924f62)","treatment of sewage sludge by anaerobic digestion, cut-off with biogenic carbon uptake | biogas | GLO",1.808420
...,...,...,...
3700,"(ecoinvent-3.10-cutoff, c8b285bf1f33640398b6775bb0ed1448)","basic oxygen furnace slag, for recovery, Recycled Content cut-off | basic oxygen furnace slag, for recovery | GLO",-0.000090
3709,"(ecoinvent-3.10-cutoff, 7ee404e8eee08f5b2d8e5121f6548bb6)","blast furnace sludge, for recovery, Recycled Content cut-off | blast furnace sludge, for recovery | GLO",-0.000319
15876,"(ecoinvent-3.10-cutoff, a225e8ab2a2ea0001ce4e479d2480a28)","sulfidic tailing, off-site, high gold content, Recycled Content cut-off | sulfidic tailing, off-site, high gold content | GLO",-0.000819


In [28]:
def extract_slack(instance):
    """
    Extracts and sorts slack values from a Pyomo model.

    Parameters:
        instance (Pyomo model): The model containing slack variables.

    Returns:
        pd.DataFrame: A DataFrame indexed by ID with slack values, sorted in descending order.
    """
    
    return pd.DataFrame(
    {'Value': [v.value for v in instance.slack.values()]},  # Extract .value from each Pyomo variable
    index=instance.slack.keys()
    ).sort_values('Value', ascending=False)

# Test the function
extract_slack(pulpo_worker.instance)

,Value
0,0.0
15714,0.0
15723,0.0
15722,0.0
15721,0.0
...,...
7856,0.0
7855,0.0
7854,0.0
7853,0.0


In [30]:
def extract_impacts(instance):
    """
    Extracts impact values and corresponding weights from the pyomo instance.

    Parameters:
        instance (Pyomo model): The resolved instance of the Pyomo model.

    Returns:
        pd.DataFrame: A DataFrame indexed by impact method with 'Value' and 'Weight', sorted by 'Weight' in descending order.
    """

    data = {'Method': [], 'Weight': [], 'Value': []}

    for i in instance.impacts.keys():
        data['Method'].append(i)
        data['Weight'].append(instance.WEIGHTS[i].value if i in instance.WEIGHTS and instance.WEIGHTS[i].value is not None else 0)
        data['Value'].append(instance.impacts[i].value if instance.impacts[i] is not None else 0)

    return pd.DataFrame(data).set_index('Method').sort_values('Weight', ascending=False)

# Test the function
extract_impacts(pulpo_worker.instance)

,Weight,Value
Method,,
"('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')",1,-0.833758


In [31]:
def extract_project_db(project, database):
    """
    Creates a DataFrame with project and database information.

    Parameters:
        project (str): Project name.
        database (str or list of str): One or more database names.

    Returns:
        pd.DataFrame: A DataFrame with 'Project' and 'Database' columns.
    """
    databases = [database] if isinstance(database, str) else database
    return pd.DataFrame({'Project': [project] * len(databases), 'Database': databases})

# Test the function
extract_project_db(project, databases)


,Project,Database
0,pulpo-ammonia,nc-inventories-ei310-all
1,pulpo-ammonia,ecoinvent-3.10-cutoff


In [34]:
def extract_choices(instance, choices, process_map, process_map_metadata):
    """
    Extracts choice results from a Pyomo model and structures them into DataFrames.

    Parameters:
        instance: Pyomo model instance containing a scaling_vector.
        choices (dict): Mapping of choices to processes. 
                        {choice_name: {process: capacity}}
        process_map (dict): Maps process keys (process.key) to process IDs.
        process_map_metadata (dict): Maps process IDs to descriptive metadata.

    Returns:
        dict: {choice_name: pd.DataFrame} 
              Each DataFrame contains:
              - "Value" (scaling factor from instance)
              - "Capacity" (capacity from choices)
              Indexed by process metadata.
    """

    results = {}
    for choice, processes in choices.items():
        data = {
            "Value": [],
            "Capacity": [],
            "Metadata": []
        }
        for process, capacity in processes.items():
            proc_id = process_map.get(process.key)
            if proc_id is None:
                continue
            data["Metadata"].append(process_map_metadata.get(proc_id, "No Metadata"))
            data["Value"].append(instance.scaling_vector[proc_id].value)
            data["Capacity"].append(capacity)

        results[choice] = pd.DataFrame(data).set_index("Metadata").sort_values("Value", ascending=False)
    
    return results

# Test the function
choices_results = extract_choices(pulpo_worker.instance, choices, pulpo_worker.lci_data['process_map'], pulpo_worker.lci_data['process_map_metadata'])

for choice, df in choices_results.items():
    print(f"Results for {choice}:")
    display(df[df["Value"] != 0])  # Filters out rows where "Value" is 0
    print("\n")


Results for biogas:


,Value,Capacity
Metadata,,
"treatment of sewage sludge by anaerobic digestion, cut-off with biogenic carbon uptake | biogas | GLO",1.80842,1.000000e+10




Results for hydrogen:


,Value,Capacity
Metadata,,




Results for heat:


,Value,Capacity
Metadata,,
heat from biomethane | heat from biomethane | GLO,13.952525,1.000000e+10




Results for biomethane:


,Value,Capacity
Metadata,,
"biogas upgrading to biomethane, chemical scrubbing | biomethane, 24 bar | GLO",0.652609,1.000000e+10
"biogas upgrading to biomethane, chemical scrubbing w/ CCS | biomethane, 24 bar | GLO",0.387563,1.000000e+10




Results for ammonia:


,Value,Capacity
Metadata,,
"ammonia production, steam methane reforming of biomethane, with CCS | ammonia production, biomethane, with CCS | GLO",1.000031,1.000000e+10


In [35]:
import pandas as pd

def extract_demand(demand):
    """
    Converts Brightway demand data into a structured DataFrame.

    Parameters:
        demand (dict): dictionary with demand data, where keys are the brightway processes and the values the demands.
    Returns:
        pd.DataFrame: Indexed by ("Reference Product", "Activity Name", "Location") with:
            - "Value" (demand amount)
    """
    data = [
        {
            "Reference Product": e.get("reference product", "Unknown"),
            "Activity Name": e.get("name", "Unknown"),
            "Location": e.get("location", "Unknown"),
            "Value": v
        }
        for e, v in demand.items()
    ]

    return pd.DataFrame(data).set_index(["Reference Product", "Activity Name", "Location"])

# Test the function
extract_demand(demand)


,,,Value
Reference Product,Activity Name,Location,
ammonia,new market for ammonia,GLO,1


In [40]:
def extract_constraints(instance, constraints, mapping, metadata, constraint_type):
    """
    Extracts scaling factors or inventory flows associated to constraints from a Pyomo model instance.

    Parameters:
        instance (Pyomo model): The Pyomo model instance.
        constraint (dict): The constraints to be extracted. These can be:
            - 'upper_limit'
            - 'lower_limit'
            - 'upper_elem_limit'
        mapping (dict): Maps process keys to IDs.
        metadata (dict): Maps IDs to metadata descriptions.
        flow_type (str): 'scaling' for process_map, 'intervention' for intervention_map.

    Returns:
        pd.DataFrame: Indexed by **ID**, containing **Key, Metadata, and Value**, sorted by Value (descending).
    """

    inverse_map = {v: k for k, v in mapping.items()}  # Reverse lookup
    
    # Select correct flow variable ('scaling' or 'intervention')
    flows = instance.scaling_vector if constraint_type == 'scaling' else instance.inv_flows if constraint_type == 'intervention' else None
    if flows is None:
        raise ValueError("Invalid flow_type. Use 'scaling' or 'intervention'.")

    # Retrieve data from flows
    data = {'ID': [], 'Key': [], 'Metadata': [], 'Value': [], 'Limit': []}
    for constraint in constraints:
        flow = mapping.get(constraint.key)
        data['ID'].append(flow)
        data['Key'].append(inverse_map.get(flow, 'Unknown'))
        data['Metadata'].append(metadata.get(flow, 'No Metadata'))
        data['Value'].append(flows[flow].value)
        data['Limit'].append(constraints.get(constraint, 'No Limit'))

    return pd.DataFrame(data).set_index('ID').sort_values('Value', ascending=False)

# Test the function
print("Upper Limit:")
print(extract_constraints(pulpo_worker.instance, upper_limit, pulpo_worker.lci_data['process_map'], pulpo_worker.lci_data['process_map_metadata'], constraint_type='scaling'))
print
print(extract_constraints(pulpo_worker.instance, lower_limit, pulpo_worker.lci_data['process_map'], pulpo_worker.lci_data['process_map_metadata'], constraint_type='scaling'))
print("Upper Element Limit:")
print(extract_constraints(pulpo_worker.instance, upper_elem_limit, pulpo_worker.lci_data['intervention_map'], pulpo_worker.lci_data['intervention_map_metadata'], constraint_type='intervention'))

Upper Limit:
                                                                Key  \
ID                                                                    
23531  (nc-inventories-ei310-all, fc378c5b9e61e417e77ba0c166897da5)   

                                                                                                                                                Metadata  \
ID                                                                                                                                                         
23531  carbon dioxide storage and transport 200 km pipeline, storage 1000 m | carbon dioxide storage and transport 200 km pipeline, storage 1000 m | GLO   

       Value  Limit  
ID                   
23531    2.0      2  
                                                                Key  \
ID                                                                    
23531  (nc-inventories-ei310-all, fc378c5b9e61e417e77ba0c166897da5)   

                        